#  Day 3: Gear Ratios

## Part 1

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [ ]:
from aocd import get_data
import random
input_data = get_data(day=3, year=2023)
for line in input_data.splitlines()[-10:]:
    print(line)

...364..149..............425............*.....662.481.......................788..$844..................506..........@.....693......&........
..........................*....408...772....+.......-.......232*......*......*.............#.+294.....*.....706................435..........
......566*972........631=.425.*...........14............984.....667..7.236....195..9.....222........757......*.....865..604&..*....344.&....
.................439..........710..............477*714..............................*544........368............395*...........435...*..204..
..........229......+..596.179........../................165.542*47........147..............653..*...................466...........119.......
.....690...@............@.......467..626........&......*.................*....=42.....724....+.816....262*.....237.@....549.383.............
......*.........750..............*..............202....967.914......279..................*........./......147.@...............*........480..
...444.......

In [ ]:
data = '''467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..'''
lines = data.splitlines()
lines

['467..114..',
 '...*......',
 '..35..633.',
 '......#...',
 '617*......',
 '.....+.58.',
 '..592.....',
 '......755.',
 '...$.*....',
 '.664.598..']

### Explain the Problem

- 2D grid of stuff (digits, symbols, periods, etc.)
- for any `(i,j)` in the grid we need to know if it is adjacent to a symbol
    - a single entry `(i,j)` is adjacent to a symbol if it satisfies atleast one of the following
        - symbol on the left or right
        - symbol above or below
        - symbol on the diagonal (top right, top left, bottom right, bottom left)
- we should create a function `def is_adjacent(i,j) -->True/False`
- maybe we store in a set each `(i,j)` if it's a symbol. Then simply check if a position in set to see if it is a symbol
- we need to extract all complete numbers from the data and for each number store its starting postition `(i,j)`
    - we can get the other positions of the digits easily from that because we will know the length of each number
    - for example in the sample data line 5 has a 58 so we would want to return that as the element `(58, ((5,7)) but we would also know  `(5,8))` is a position too.
    - And we could mark the **number** as adjacent if any of its corresponding digit positions are **adjacent**

In [ ]:
def get_symbol_locs(lines: list[str]):
    symbol_locs = set()
    for i,line in enumerate(lines):
        for j, element in enumerate(line):
            if not (element.isdigit() or element == '.'):
                symbol_locs.add((i,j))
    return symbol_locs
symbol_locs = get_symbol_locs(lines)
print(symbol_locs)

{(5, 5), (4, 3), (8, 3), (3, 6), (1, 3), (8, 5)}


- We have a function which gives us a set of all the symbol locations `(i, j)`.
- We can use this to check if any position `(i,j)` is a symbol by checking if it's in the set

Next let's look into figuring out the adjacent problem

In [ ]:
for l in lines:
    print(l)

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..


In [ ]:
def is_adjacent(i, j, symbol_locs):
    if any(
        [
            (i, j+1) in symbol_locs, # right
            (i, j-1) in symbol_locs, # left
            (i-1, j) in symbol_locs, # top
            (i+1, j) in symbol_locs, # bottom
            (i-1, j+1) in symbol_locs, # top right
            (i-1, j-1) in symbol_locs, # top left
            (i+1, j+1) in symbol_locs, # bottom right
            (i+1, j-1) in symbol_locs, # bottom left
        ]
    ):
        return True
    return False
assert is_adjacent(0,2,symbol_locs)
assert is_adjacent(0,3,symbol_locs)
assert is_adjacent(2,6,symbol_locs)
assert is_adjacent(9,3,symbol_locs)
assert is_adjacent(9,2,symbol_locs)
assert is_adjacent(9,6,symbol_locs)
assert is_adjacent(6,4,symbol_locs)

assert not is_adjacent(9,9,symbol_locs)
assert not is_adjacent(6,3,symbol_locs)

- now we have the `is_adjacent` function which tells us if `(i,j)` is adjacent to a symbol
- the next step is to extract all the numbers and their list of positions making up those digits in the number

In [ ]:
lines

['467..114..',
 '...*......',
 '..35..633.',
 '......#...',
 '617*......',
 '.....+.58.',
 '..592.....',
 '......755.',
 '...$.*....',
 '.664.598..']

**AI Prompt**

I want to use some sort of regex to extract the numbers from a line as well as its starting position.
I think the simplest thing would to have a little function that takes the `line` string as input. 
I already wrote some code that loops over the rows and columns and I mostly worked. But it may have some issues in terms
of edgecases like numbers apearing at the end of a line.

Original Code - I don't want to use it and am looking for something simpler.
```python
numbers = []

number = ''
number_positions = []
for i,line in enumerate(lines):
    for j, element in enumerate(line):
        if element.isdigit():
            number += element
            number_positions.append((i,j))
        else:
            if number:
                numbers.append([(int(number), number_positions)])
            # reset
            number = ''
            number_positions = []
numbers
```

So I think using `re` would be better right?

**AI Response**

Yes, using regex would be a cleaner approach! Let me help you think through this. 

First, would you like me to explain how we could use regex to find numbers in a string? We'll need to capture both the number and its position. I can suggest a basic pattern to start with, and then we can work on getting the position information.

Would you like me to explain the regex approach step by step, or would you prefer to try writing it yourself first with some hints?

**AI Prompt**

I have always sucked at regex but know a little like `/d.*` means a digit followed by anything? But assume I know little or nothing of `re`.

**AI Response**

No problem! Let's build this up step by step. First, let's clarify a few regex basics:

- `\d` matches a single digit (0-9)
- `\d+` matches one or more digits in a row
- `re.finditer()` will give us both the matches AND their positions

Would you like me to show you how to use `re.finditer()` with a simple example using one line of our data? Then we can build on that to get exactly what we need.

**AI Prompt**

Yes I want both the number and its starting position. I'm going to need to see an example of its use.

```python
import re
line = '1...664.598....2...123'
re.finditer('\d+', line)
```
??

**AI Response**

Let me show you how to use `finditer`. When you run that code, it creates an iterator of match objects. Let's see what information we can get from each match:

```python
import re
line = '1...664.598....2...123'
for match in re.finditer(r'\d+', line):
    number = int(match.group())  # The actual number
    start_pos = match.start()    # Starting position in string
    print(f"Found number {number} at position {start_pos}")
```

Try running this - it should show you each number and where it was found. Would you like me to explain what each part does?

In [ ]:
import re
line = '1...664.598....2...123'
for match in re.finditer(r'\d+', line):
    number = int(match.group())  # The actual number
    start_pos = match.start()    # Starting position in string
    print(f"Found number {number} at position {start_pos}")

Found number 1 at position 0
Found number 664 at position 4
Found number 598 at position 8
Found number 2 at position 15
Found number 123 at position 19


**AI Prompt**

Awesome. Exactly what I need. So it's called `finditer` because it iteratively finds each match?

**AI Response**

Yes, exactly! While `re.find()` would just give you the first match, `finditer` gives you an iterator that lets you go through all matches one by one. That's why it works so nicely in a `for` loop.

Each match object contains useful information like:
- `match.group()`: the actual text that matched
- `match.start()`: where the match begins
- `match.end()`: where the match ends

I think that is enough for you to continue on your solution.

In [ ]:
for l in lines:
    print(l)
    
numbers = []
for i, line in enumerate(lines):
    numbers.extend([(int(match.group()), (i, match.start())) for match in  re.finditer('\d+', line)])
print(numbers)

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
[(467, (0, 0)), (114, (0, 5)), (35, (2, 2)), (633, (2, 6)), (617, (4, 0)), (58, (5, 7)), (592, (6, 2)), (755, (7, 6)), (664, (9, 1)), (598, (9, 5))]


In [ ]:
def extract_numbers(lines): 
    numbers = []
    for i, line in enumerate(lines):
        numbers.extend([(int(match.group()), (i, match.start())) for match in  re.finditer('\d+', line)])
    return numbers
numbers = extract_numbers(lines)
assert numbers == [(467, (0, 0)), (114, (0, 5)), (35, (2, 2)), (633, (2, 6)), (617, (4, 0)), (58, (5, 7)), (592, (6, 2)), (755, (7, 6)), (664, (9, 1)), (598, (9, 5))]
numbers

[(467, (0, 0)),
 (114, (0, 5)),
 (35, (2, 2)),
 (633, (2, 6)),
 (617, (4, 0)),
 (58, (5, 7)),
 (592, (6, 2)),
 (755, (7, 6)),
 (664, (9, 1)),
 (598, (9, 5))]

- Now we have the function to extract all the numbers and starting positions of each number
- Now we just need to loop over those numbers and check if any of the digits are adjacent.

In [ ]:
for n, start_pos in numbers:
    print(n, start_pos)

467 (0, 0)
114 (0, 5)
35 (2, 2)
633 (2, 6)
617 (4, 0)
58 (5, 7)
592 (6, 2)
755 (7, 6)
664 (9, 1)
598 (9, 5)


In [ ]:
for n, start_pos in numbers:
    positions = [start_pos] + [(start_pos[0], start_pos[1] + j) for j in range(1, len(str(n)))]
    print(n, positions)

467 [(0, 0), (0, 1), (0, 2)]
114 [(0, 5), (0, 6), (0, 7)]
35 [(2, 2), (2, 3)]
633 [(2, 6), (2, 7), (2, 8)]
617 [(4, 0), (4, 1), (4, 2)]
58 [(5, 7), (5, 8)]
592 [(6, 2), (6, 3), (6, 4)]
755 [(7, 6), (7, 7), (7, 8)]
664 [(9, 1), (9, 2), (9, 3)]
598 [(9, 5), (9, 6), (9, 7)]


In [ ]:
for n, start_pos in numbers:
    positions = [start_pos] + [(start_pos[0], start_pos[1] + j) for j in range(1, len(str(n)))]
    if any([is_adjacent(p[0], p[1], symbol_locs) for p in positions]):
        print(n)

467
35
633
617
592
755
664
598


In [ ]:
part_numbers = []
for n, start_pos in numbers:
    positions = [start_pos] + [(start_pos[0], start_pos[1] + j) for j in range(1, len(str(n)))]
    if any([is_adjacent(p[0], p[1], symbol_locs) for p in positions]):
        part_numbers.append(n)
sum(part_numbers)

4361

In [ ]:
def sum_part_numbers(lines, symbol_locs):
    numbers = extract_numbers(lines)
    part_numbers = []
    for n, start_pos in numbers:
        positions = [start_pos] + [(start_pos[0], start_pos[1] + j) for j in range(1, len(str(n)))]
        if any([is_adjacent(p[0], p[1], symbol_locs) for p in positions]):
            part_numbers.append(n)
    return sum(part_numbers)

ans = sum_part_numbers(lines, symbol_locs)
assert ans == 4361
print(ans)

4361


## Final Answer

In [ ]:
lines = input_data.splitlines()
symbol_locs = get_symbol_locs(lines)
sum_part_numbers(lines, symbol_locs)

556057

## Part 2

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [ ]:
lines = data.splitlines()
for l in lines:
    print(l)

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..


In [ ]:
[(i,line) for i,line in enumerate(lines)]

[(0, '467..114..'),
 (1, '...*......'),
 (2, '..35..633.'),
 (3, '......#...'),
 (4, '617*......'),
 (5, '.....+.58.'),
 (6, '..592.....'),
 (7, '......755.'),
 (8, '...$.*....'),
 (9, '.664.598..')]

In [ ]:
star_locs = [(i,j) for i,line in enumerate(lines) for j, symbol in enumerate(line) if symbol == '*']
star_locs

[(1, 3), (4, 3), (8, 5)]

In [ ]:
extract_numbers(lines)

[(467, (0, 0)),
 (114, (0, 5)),
 (35, (2, 2)),
 (633, (2, 6)),
 (617, (4, 0)),
 (58, (5, 7)),
 (592, (6, 2)),
 (755, (7, 6)),
 (664, (9, 1)),
 (598, (9, 5))]

In [ ]:
for n, start_pos in numbers:
    positions = [start_pos] + [(start_pos[0], start_pos[1] + j) for j in range(1, len(str(n)))]
    print(n, positions)

467 [(0, 0), (0, 1), (0, 2)]
114 [(0, 5), (0, 6), (0, 7)]
35 [(2, 2), (2, 3)]
633 [(2, 6), (2, 7), (2, 8)]
617 [(4, 0), (4, 1), (4, 2)]
58 [(5, 7), (5, 8)]
592 [(6, 2), (6, 3), (6, 4)]
755 [(7, 6), (7, 7), (7, 8)]
664 [(9, 1), (9, 2), (9, 3)]
598 [(9, 5), (9, 6), (9, 7)]


In [ ]:
def get_number_positions(lines):
    numbers = extract_numbers(lines)
    number_positions = []
    for n, start_pos in numbers:
        positions = [start_pos] + [(start_pos[0], start_pos[1] + j) for j in range(1, len(str(n)))]
        number_positions.append((n, positions))
    return number_positions
number_positions = get_number_positions(lines)
number_positions

[(467, [(0, 0), (0, 1), (0, 2)]),
 (114, [(0, 5), (0, 6), (0, 7)]),
 (35, [(2, 2), (2, 3)]),
 (633, [(2, 6), (2, 7), (2, 8)]),
 (617, [(4, 0), (4, 1), (4, 2)]),
 (58, [(5, 7), (5, 8)]),
 (592, [(6, 2), (6, 3), (6, 4)]),
 (755, [(7, 6), (7, 7), (7, 8)]),
 (664, [(9, 1), (9, 2), (9, 3)]),
 (598, [(9, 5), (9, 6), (9, 7)])]

In [ ]:
for star_loc in star_locs:
    part_numbers = []
    for n, positions in number_positions:
        if any([is_adjacent(star_loc[0], star_loc[1], positions)]):
            part_numbers.append(n)
        if len(part_numbers) > 2:
            break
    if len(part_numbers) == 2:
        print(star_loc, part_numbers)

(1, 3) [467, 35]
(8, 5) [755, 598]


In [ ]:
import math

def get_gear_ratio(lines):
    star_locs = [(i,j) for i,line in enumerate(lines) for j, symbol in enumerate(line) if symbol == '*']
    number_positions = get_number_positions(lines)
    gear_ratio = 0
    for star_loc in star_locs:
        part_numbers = []
        for n, positions in number_positions:
            if any([is_adjacent(star_loc[0], star_loc[1], positions)]):
                part_numbers.append(n)
            if len(part_numbers) > 2:
                break
        if len(part_numbers) == 2:
            gear_ratio += math.prod(part_numbers)
    return gear_ratio

get_gear_ratio(lines)

467835

## Final Answer

In [ ]:
lines = input_data.splitlines()
get_gear_ratio(lines)

82824352